Alunos:

- Bruno Santos (bfss@cesar.school)
- Caio Guedes (ccsg@cesar.school)

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install langchain langchain_openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00


In [12]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.5 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
response = llm.invoke("Qual a previsão do tempo em Recife-RN?")
print(response.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a previsão do tempo. Recomendo que você consulte um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas sobre o tempo em Recife-RN.


In [5]:
a = 18273840567
b = 12830192835

response = llm.invoke("Quanto é 18273840567*12830192835? Responda apenas a o resultado.")
print(response.content)
print()
print(f'Resposta esperada:', a*b)

234084204189203999595.

Resposta esperada: 234456898310655737445


In [6]:
import random
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Soma dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a * b

@tool
def get_weather(city: str) -> str:
    """Retorna um clima aleatório para uma cidade.

    Args:
        city: Nome da cidade
    """
    weather = random.choice(["ensolarado", "nublado", "chuvoso", "tempestuoso", "neve"])
    return weather

tools = [add, multiply, get_weather]
llm_with_tools = llm.bind_tools(tools)

In [7]:
get_weather.invoke({"city": "Recife-RN"})

'chuvoso'

In [8]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")]

ai_msg = llm_with_tools.invoke(messages)

print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'city': 'Recife-RN'}, 'id': 'call_SvMJKkJRB9yOlRTbCctubPk3', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 123, 'b': 321}, 'id': 'call_hnO6SbirfosaAeeAmYQQ76NH', 'type': 'tool_call'}]


In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [14]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?")
    ]
})

print(response["messages"])

[HumanMessage(content='Qual a previsão do tempo em Recife-RN? E também quanto é 123*321?', additional_kwargs={}, response_metadata={}, id='10fa1a5a-0f49-403c-a0d4-8897e8b1df32'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2kJ5vC3tME3HIdkwiC8D4xaJ', 'function': {'arguments': '{"city": "Recife-RN"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_qLfswAgmUm4srFF7iUaLOZgh', 'function': {'arguments': '{"a": 123, "b": 321}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 145, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-794c391e-b7da-4e47-80c9-3d4e3a32

In [15]:
print(response["messages"][-1].content)

A previsão do tempo em Recife-RN é de céu nublado. 

Além disso, o resultado de 123 * 321 é 39.483.


## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [17]:
@tool
def criar_arquivo(nome_arquivo: str, conteudo: str) -> str:
    """Cria um arquivo de texto com o conteúdo especificado.

    Args:
        nome_arquivo: Nome do arquivo (incluindo extensão).
        conteudo: Conteúdo a ser escrito no arquivo.
    """
    with open(nome_arquivo, "w") as f:
        f.write(conteudo)
    return f"Arquivo '{nome_arquivo}' criado com sucesso!"

In [22]:
# Adicionar a nova ferramenta à lista
tools.append(criar_arquivo)

# Atualizar o agente executor com a nova ferramenta
agent_executor = create_react_agent(llm, tools)

In [23]:
response = agent_executor.invoke({
    "messages": [HumanMessage("Crie um arquivo chamado 'teste.txt' com o conteúdo 'Mensagem de Teste 1, 2, 3'")]
})

# Verificar a resposta
print(response["messages"])
print(response["messages"][-1].content)

# Confirmar se o arquivo foi criado
if os.path.exists("teste.txt"):
    with open("teste.txt", "r") as f:
        print("Conteúdo do arquivo criado:")
        print(f.read())


[HumanMessage(content="Crie um arquivo chamado 'teste.txt' com o conteúdo 'Mensagem de Teste 1, 2, 3'", additional_kwargs={}, response_metadata={}, id='6998fb60-86ed-4b99-8eb6-1733b56bf2aa'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i2bfVGKpaYh1Lw7BLRtdusns', 'function': {'arguments': '{"nome_arquivo":"teste.txt","conteudo":"Mensagem de Teste 1, 2, 3"}', 'name': 'criar_arquivo'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 265, 'total_tokens': 301, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee4d41a2-ac69-40ec-aede-3fbd87a52660-0', tool_calls=[{'name': 'criar_arquivo', 'args': {'nome_arquivo':

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [24]:
# Definição da ferramenta
@tool
def contar_letra(palavra: str, letra: str) -> int:
    """Conta o número de ocorrências de uma letra em uma palavra.

    Args:
        word: A palavra em que a contagem será feita.
        letter: A letra a ser contada.

    Returns:
        O número de ocorrências da letra na palavra.
    """
    if len(letra) != 1:
        return "Por favor, insira apenas uma letra."
    return palavra.count(letra)

In [26]:
# Adicionar a ferramenta à lista
tools = [contar_letra]

# Criar o agente
agent_executor = create_react_agent(llm, tools)

# Testar o agente
test_message = HumanMessage("Conte quantas vezes a letra 'o' aparece na palavra 'pneumoultramicroscopicossilicovulcanoconiótico'.")

response = agent_executor.invoke({
    "messages": [test_message]
})

# Verificar a resposta
print(response["messages"])
print(response["messages"][-1].content)

[HumanMessage(content="Conte quantas vezes a letra 'o' aparece na palavra 'pneumoultramicroscopicossilicovulcanoconiótico'.", additional_kwargs={}, response_metadata={}, id='2b4dedc4-e5e6-4dd9-bf6f-028b96d6c5be'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_238Eg0mOOB7EzCA2rRUue2Sk', 'function': {'arguments': '{"palavra":"pneumoultramicroscopicossilicovulcanoconiótico","letra":"o"}', 'name': 'contar_letra'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 119, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0a435075-01ba-4684-ab67-02a0eb26679e-0', tool_calls=[{'name': 'contar_letra',